In [15]:
import os
import pandas as pd
from collections import Counter

def extrair_parametros(path_params):
    parametros = {"pop_size": None, "ngen": None, "refresh_fraction": None}
    if os.path.exists(path_params):
        with open(path_params, "r") as f:
            for linha in f:
                for chave in parametros:
                    if linha.strip().startswith(f"{chave}:"):
                        parametros[chave] = linha.split(":")[1].strip()
    return parametros

def processar_runs_em_outputs(diretorio_outputs="outputs", top_n_linhas=10, top_n_genes=10):
    tabelas = []

    # Caminhar dois níveis: outputs/PASTA/PASTA/final.csv
    for sub1 in os.listdir(diretorio_outputs):
        path_sub1 = os.path.join(diretorio_outputs, sub1)
        if not os.path.isdir(path_sub1):
            continue
        for sub2 in os.listdir(path_sub1):
            path_sub2 = os.path.join(path_sub1, sub2)
            if not os.path.isdir(path_sub2):
                continue

            final_path = os.path.join(path_sub2, "final.csv")
            params_path = os.path.join(path_sub2, "params.txt")

            if os.path.isfile(final_path):
                try:
                    df = pd.read_csv(final_path)
                    df_top = df.sort_values(by="Fitness", ascending=False).head(top_n_linhas)

                    todos_genes = []
                    for linha in df_top["Knocked Genes"]:
                        todos_genes.extend(linha.split(","))

                    top_genes = Counter(todos_genes).most_common(top_n_genes)
                    df_genes = pd.DataFrame(top_genes, columns=["Gene", "Frequência"])

                    nome_run = os.path.basename(path_sub2)
                    parametros = extrair_parametros(params_path)
                    titulo = (
                        f"Run: {nome_run} | "
                        f"pop_size={parametros['pop_size']}, "
                        f"ngen={parametros['ngen']}, "
                        f"refresh_fraction={parametros['refresh_fraction']}"
                    )

                    tabelas.append((titulo, df_genes))
                except Exception as e:
                    print(f"Erro ao processar {path_sub2}: {e}")

    return tabelas

tabelas = processar_runs_em_outputs("outputs")

for titulo, df in tabelas:
    print("=" * len(titulo))
    print(titulo)
    print("=" * len(titulo))
    print(df)
    print("\n")
